# Data handling

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/nhkim55/bigdata_fintech_python/blob/main/code/ch10_categorical.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/nhkim55/bigdata_fintech_python/blob/main/code/ch10_categorical.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

# 범주형 데이터 다루기

* 관측 대상을 양이 아니라 질로 측정하는 것이 유용할 경우에 범주를 가지는 정성적 데이터(qualitative data)를 이용
* 명목형(nominal) 범주
  * 파랑, 빨강, 초록
  * 남자, 여자
  * 바나나, 딸기, 사과
* 순서형(ordinal) 범주
  * 낮음, 중간, 높음
  * 청년, 노인
  * 동의, 중립, 반대 
* 범주형 정보는 데이터에서 종종 벡터나 문자열의 열로 표현됨
* 대부분의 머신러닝 알고리즘에는 수치값을 입력해야 하므로 문제가 됨

* 예를 들어 k-최근접 이웃에서는 샘플 간의 거리를 계산할 때 유클리드 거리가 사용됨

$$\sqrt{\sum^n_{i=1}(x_i-y_i)^2}$$

여기에서 x와 y는 두 개의 샘플이고 아래 첨자 $i$는 샘플의 $i$번째 특성값을 나타냄
  * $x_i$의 값이 문자열이라면 거리 계산이 불가능 
  * 문자열을 어떤 수치형태로 바꾸어 유클리드 거리 공식에 넣어야 함
  * 범주에 있는 정보가 적절히 인코딩되도록 변환하는 것이 중요(순서, 범주 사이의 상대적 간격 등)



## 순서가 없는 범주 특성 인코딩하기

* 순서를 가지지 않는 클래스로 이루어진 명목형 특성
* 사이킷런의 LabelBinarizer를 사용하여 특성을 원-핫 인코딩함


In [ ]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# 특성을 만듭니다.
feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"],
                    ["Texas"]])

# 원-핫 인코더를 만듭니다.
one_hot = LabelBinarizer()

# 특성을 원-핫 인코딩합니다.
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

classes_ 속성에서 클래스를 확인할 수 있음

In [ ]:
# 특성의 클래스를 확인합니다.
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

원-핫 인코딩을 되돌리려면 inverse_transform 메서드를 사용

In [ ]:
# 원-핫 인코딩을 되돌립니다.
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

판다스의 get_dummies 를 사용해서 특성을 원-핫 인코딩할 수도 있음

In [ ]:
# 라이브러리를 임포트합니다.
import pandas as pd

# 특성으로 더미(dummy) 변수를 만듭니다.
pd.get_dummies(feature[:,0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


사이킷런에 있는 한 가지 유용한 기능은 샘플이 여러 개의 클래스를 가지고 있는 경우를 다룰 수 있음

In [ ]:
# 다중 클래스 특성을 만듭니다.
multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delware", "Florida"),
                      ("Texas", "Alabama")]

# 다중 클래스 원-핫 인코더를 만듭니다.
one_hot_multiclass = MultiLabelBinarizer()

# 다중 클래스 특성을 원-핫 인코딩합니다.
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

여기서도 classes_ 속성으로 클래스를 확인할 수 있음

In [ ]:
# 클래스를 확인합니다.
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

* 각 클래스를 하나의 수치값에 할당하는 것이 적절한 방법이라고 생각할 수 있음(예를 들어 Texas=1, California=2). 하지만 클래스가 태생적으로 순서를 가지고 있지 않다면 이 수치값은 존재하지 않는 순서를 잘못 만들게 됨
*  올바른 방법은 원본 특성에 있는 클래스마다 이진 특성을 하나씩 만드는 것(원-핫 인코딩 또는 더미 인코딩(dummy encoding))
  * 예제처럼 세개의 클래스를 가지고 있을 경우(Texas, California, Delaware), 원-핫 인코딩을 하면 각 클래스마다 하나의 특성이 만들어짐
  * 샘플의 클래스에 해당하는 특성은 1이되고 나머지 특성은 0
  * 클래스에 순서가 없다는 개념을 그대로 유지하면서 샘플의 클래스 소속을 표현할 수 있음
* 특성을 원-핫 인코딩한 후에는 선형 의존성을 피하기 위해 결과 행렬에서 원-핫 인코딩된 특성 중 하나를 삭제하는 것이 좋음

* [더미 변수의 함정](https://www.algosome.com/articles/dummy-variable-trap-regression.html)
* [원-핫 인코딩을 사용할 때 열 삭제](https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding)

### 붙임

* LabelBinarizer는 문자열 타깃 데이터를 원-핫 인코딩으로 변환할 때 사용
* 문자열 타깃 데이터를 정수 레이블로 변환할 때는 LabelEncoder를 사용
* LabelBinarizer나 LabelEncoder는 일차원 배열을 입력으로 받음

* 사이킷런의 OneHotEncoder 클래스가 정수형  특성과 문자열 데이터를 원-핫 인코딩으로 변환
* OneHotEncoder 클래스는 기본적으로 희소 배열을 반환
* sparse=False로 지정하면 밀집 배열을 얻을 수 있음

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
# 여러 개의 열이 있는 특성 배열을 만듭니다.
feature = np.array([["Texas", 1],
                    ["California", 1],
                    ["Texas", 3],
                    ["Delaware", 1],
                    ["Texas", 1]])

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit_transform(feature)

array([[0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

* Californial, Delaware, Texas가 처음 세 개의 열에 원-핫 인코딩되었고 1과 3이 나머지 두 개의 열에 원-핫 인코딩되었음
* 정수도 문자열처럼 취급하여 변환됨
* Categories_ 속성으로 클래스를 확인

In [ ]:
one_hot_encoder.categories_

[array(['California', 'Delaware', 'Texas'], dtype='<U10'),
 array(['1', '3'], dtype='<U10')]

* OneHotEncoder는 입력 특성 배열을 모두 범주형으로 인식하여 변환
* 특정 열에만 적용하려면 ColumnTransformer와 함께 사용 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

# 여러 개의 열이 있는 특성 배열을 만듭니다.
feature = np.array([["Texas", 1],
                    ["California", 1],
                    ["Texas", 3],
                    ["Delaware", 1],
                    ["Texas", 1]])
df = pd.DataFrame(feature, columns=["f1", "f2"])

# (이름, 변환기, 열 리스트)로 구성된 튜플의 리스트를 ColumnTransformer에 전달합니다.
ct = ColumnTransformer([
                        ("OneHot", OneHotEncoder(sparse=False), ['f1']),
                        (("pass",'passthrough', ['f2'] ))
                        ])

ct.fit_transform(df)

array([[0.0, 0.0, 1.0, '1'],
       [1.0, 0.0, 0.0, '1'],
       [0.0, 0.0, 1.0, '3'],
       [0.0, 1.0, 0.0, '1'],
       [0.0, 0.0, 1.0, '1']], dtype=object)

## 순서가 있는 범주형 특성 인코딩하기

* 순서정보를 포함한 순서형 범주형 변수(예를 들어 high, medium, low)
* 판다스 데이터프레임의 replace 메서드를 사용하여 문자열 레이블을 수치값으로 변환

In [ ]:
# 라이브러리를 임포트합니다.
import pandas as pd

# 특성을 만듭니다.
dataframe = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})

# 매핑 딕셔너리를 만듭니다.
scale_mapper = {"Low":1,
                "Medium":2,
                "High":3}

# 특성을 정수로 변환합니다.
dataframe["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

* 특성 클래스에 자연적으로 어떤 순서가 포함된 경우 
* 대표적인 예는 리커트 척도(Likert scale)
  * 매우 그렇다, 그렇다, 보통이다, 그렇지 않다, 전혀 그렇지 않다
* 머신러닝에 사용할 특성을 인코딩할 때 순서가 있는 클래스는 순서 개념을 가진 수치값으로 변환해야 함
* 가장 자주 사용하는 방법은 클래스 레이블 문자열을 정수로 매핑하는 딕셔너리를 만들고 이를 필요한 특성에 적용하는 것

* 어떤 수치값을 선택하는지는 클래스에 내재된 순서 정보에 기반
* 예제에서는 클래스간에 동일한 간격을 사용. high가 low의 세 배 더 큰 값을 가짐
* 경우에 따라 이런 설정이 잘 맞을 수 있지만 클래스사이 간격이 동일하지 않다면 문제가 됨
 

In [ ]:
dataframe = pd.DataFrame({"Score": ["Low",
                                    "Low",
                                    "Medium",
                                    "Medium",
                                    "High",
                                    "Barely More Than Medium"]})

scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 3,
                "High":4}

dataframe["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    4
5    3
Name: Score, dtype: int64

* 위의 예제에서 Low와 Medium 사이의 거리가 Medium과 Barely More Than Medium 사이의 거리와 같지만 실제로는 그렇지 않음
* 클래스에 매핑하는 수치값에 클래스 간 거리를 고려하여 결정


In [ ]:
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 2.1,
                "High":3}
dataframe["Score"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: Score, dtype: float64

### 붙임

* 사이킷런의 순서형 데이터를 정수로 인코딩하는 OrdinalEncoder
* 클래스 범주를 순서대로 변환
* 정수 데이터도 범주형으로 인식하여 변환

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

df = pd.DataFrame({'f1':['high', 'low', 'medium'], 'f2':[10, 3, 50] })

ordinal_encoder = OrdinalEncoder(categories=[['low', 'medium', 'high'],sorted(df['f2'].unique())])
ordinal_encoder.fit_transform(df)

array([[2., 1.],
       [0., 0.],
       [1., 2.]])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

df = pd.DataFrame({'f1':['high', 'low', 'medium'], 'f2':[10, 3, 50] })

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(df)

array([[0., 1.],
       [1., 0.],
       [2., 2.]])

In [ ]:
ordinal_encoder.categories_

[array(['low', 'medium', 'high'], dtype=object), array([ 3, 10, 50])]

## 특성의 딕셔너리를 인코딩하기

* 사이킷런의 DictVectorizer를 사용

In [ ]:
# 라이브러리를 임포트합니다.
from sklearn.feature_extraction import DictVectorizer

# 딕셔너리를 만듭니다.
data_dict = [{"Red": 2, "Blue": 4},
             {"Red": 4, "Blue": 3},
             {"Red": 1, "Yellow": 2},
             {"Red": 2, "Yellow": 2}]

# DictVectorizer 객체를 만듭니다.
dictvectorizer = DictVectorizer(sparse=False)

# 딕셔너리를 특성 행렬로 변환합니다.
features = dictvectorizer.fit_transform(data_dict)

# 특성 행렬을 확인합니다.
print(features)

[[4. 2. 0.]
 [3. 4. 0.]
 [0. 1. 2.]
 [0. 2. 2.]]


* 기본적으로 DictVectorizer는 0이 아닌 값의 원소만 저장하는 희소행렬을 반환
* 이는 매우 큰 행렬을 다루어야 할 때 메모리 사용량을 최소화하는데 도움이 됨(자연어 처리 분야에서는 흔한 일)
* DictVectorizer를 sparse=False로 지정하면 밀집 벡터를 출력

get_feature_names 메서드를 사용하여 생성된 특성의 이름을 얻을 수 있음

In [ ]:
# 특성 이름을 얻습니다.
feature_names = dictvectorizer.get_feature_names()

# 특성 이름을 확인합니다.
feature_names

['Blue', 'Red', 'Yellow']

필수적이진 않지만 미려한 출력을 위해 판다스 데이터프레임으로 출력할 수 있음

In [ ]:
# 라이브러리를 임포트합니다.
import pandas as pd

# 특성으로 데이터프레임을 만듭니다.
pd.DataFrame(features, columns=feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


* 딕셔너리는 많은 프로그래밍 언어에서 즐겨 사용하는 데이터 구조
* 하지만 머신러닝 알고리즘은 행렬 형태의 데이터를 기대
* 이를 위해 사이킷런의 DictVectorizer 사용


* 이런 작업은 자연어 처리 분야에서 자주 발생
* 예를 들어 문서 데이터를 가지고 있을 때 각 문서에 등장한 모든 단어의 횟수를 담은 딕셔너리를 만들 수 있음
* dictvectorizer를 사용하면 각 문서에 등장한 단어 횟수를 특성으로 하는 특성 행렬을 만들 수 있음

In [ ]:
# 네 개의 문서에 대한 단어 카운트 딕셔너리를 만듭니다.
doc_1_word_count = {"Red": 2, "Blue": 4}
doc_2_word_count = {"Red": 4, "Blue": 3}
doc_3_word_count = {"Red": 1, "Yellow": 2}
doc_4_word_count = {"Red": 2, "Yellow": 2}

# 리스트를 만듭니다.
doc_word_counts = [doc_1_word_count,
                   doc_2_word_count,
                   doc_3_word_count,
                   doc_4_word_count]

# 단어 카운트 딕셔너리를 특성 행렬로 변환합니다.
dictvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

* 위 예제는 고유한 단어가 세 개(Red, Yellow, Blue) 뿐이므로 행렬에 세 개의 특성만 있음
* 문서가 대학 도서관에 잇는 어떤 책이라면 아주 큰 특성 행렬이 만들어짐(이 경우에는 sparse 매개변수를 True로 설정)

## 누락된 클래스 값을 대체하기

* 범주형 특성에 있는 누락된 값을 예측된 값으로 바꾸고자 할 때 이상적인 해결은 머신러닝 **분류 알고리즘**을 훈련하여 누락된 값을 예측하는 것
* 일반적으로 k-최근접 이웃(KNN) 분류기 사용

In [ ]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 범주형 특성을 가진 특성 행렬을 만듭니다.
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

# 범주형 특성에 누락된 값이 있는 특성 행렬을 만듭니다.
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# KNN 학습기를 훈련합니다.
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:,0])

# 누락된 값의 클래스를 예측합니다.
imputed_values = trained_model.predict(X_with_nan[:,1:])

# 예측된 클래스와 원본 특성을 열로 합칩니다.
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 두 특성 행렬을 연결합니다.
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

* 다른 방법은 누락된 값을 특성에서 가장 자주 등장하는 값으로 채우는 것

In [ ]:
from sklearn.impute import SimpleImputer

# 두 개의 특성 행렬을 합칩니다.
X_complete = np.vstack((X_with_nan, X))

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

* 범주형 특성에 누락된 값이 있을 때 가장 좋은 방법은 머신러닝 알고리즘으로 누락된 값을 예측하는 것
* 누락된 값이 있는 특성을 타깃으로 하고 다른 특성을 특성행렬로 사용할 수 있음
* 많이 사용하는 알고리즘은 KNN으로 k 최근접 이웃의 다수 클래스를 누락된 값에 할당


* 또는 특성에서 가장 자주 등장하는 클래스를 누락된 값으로 예측할 수 있음
* KNN보다 덜 정교하지만 대규모 데이터에 적용하기 훨씬 쉬움
* 두 경우 모두 대체된 값이 있는 샘플인지 나타내는 이진 특성을 추가하는 것을 권장 

* [랜덤 포레스트(random forest) 분류기에서 누락된 값 극복하기](https://medium.com/airbnb-engineering/overcoming-missing-values-in-a-random-forest-classifier-7b1fc1fc03ba)
* [대체 방법으로서 k-최근접 이웃 연구](https://sites.icmc.usp.br/gbatista/files/his2002.pdf)

## 불균형한 클래스 다루기

* 타깃 벡터가 매우 불균형한 클래스로 이루어진 경우
* 근본적인 해결책은 더 많은 데이터를 모으는 것이나 불가능하다면 모델 평가 지표를 바꿀 수 있음
* 잘 동작하지 않는다면 모델에 내장된 클래스 가중치 매개변수를 사용하거나 다운샘플링이나 업샘플링을 고려해 볼 수 있음
* 여기에서는 클래스 가중치 매개변수, 다운샘플링, 업샘플링에 초점을 맞춤

* 예제를 위해 클래스가 불균형한 데이터 준비
* 피셔의 붓꽃 데이터셋은 붓꽃의 품종을 나타내는 세 개 클래스(Iris setosa, Iris virginica, Iris versicolor)의 샘플을 50개씩 고르게 가지고 있음
* 불균형 데이터셋을 만들기 위해 Iris setosa 샘플 50개 중 40개를 삭제
* Iris virginica와 Iris versicolor 클래스를 합침
* Iris setosa 샘플(클래스 0) 10개와 Iris setosa가 아닌 샘플(클래스 1) 100개로 이루어진 불균형 데이터 생성

In [ ]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# 붓꽃 데이터를 적재합니다.
iris = load_iris()

# 특성 행렬을 만듭니다.
features = iris.data

# 타깃 벡터를 만듭니다.
target = iris.target

# 처음 40개 샘플을 삭제합니다.
features = features[40:,:]
target = target[40:]

# 클래스 0을 음성 클래스로 하는 이진 타깃 벡터를 만듭니다.
target = np.where((target == 0), 0, 1)

# 불균형한 타깃 벡터를 확인합니다.
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

* 사이킷런에 있는 많은 알고리즘은 훈련할 때 불균형한 영향을 줄일 수 있도록 클래스에 가중치를 부여할 수 있는 매개변수를 제공
* RandomForestClassifier는 class_weight 매개변수를 가진 인기 높은 분류 알고리즘
* 매개변수값에 원하는 클래스 가중치를 직접 지정할 수 있음

In [ ]:
# 가중치를 만듭니다.
weights = {0: .9, 1: 0.1}

# 가중치를 부여한 랜덤 포레스트 분류기를 만듭니다.
RandomForestClassifier(class_weight=weights)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 0.9, 1: 0.1}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

* 또는 balanced로 지정하여 클래스 빈도에 반비례하게 자동으로 가중치를 만들 수 있음

In [ ]:
# 균형잡힌 클래스 가중치로 랜덤 포레스트 모델을 훈련합니다.
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

* 다수 클래스의 샘플을 줄이거나(다운샘플링) 소수 클래스의 샘플을 늘릴 수도 있음(업샘플링)
* 다운샘플링에서는 다수 클래스(즉, 더 많은 샘플을 가진 클래스)에서 중복을 허용하지 않고 랜덤하게 샘플을 선택하여 소수 클래스와 같은 크기의 샘플 부분집합을 만들어 줌
  * 예를 들면, 소수 클래스에 10개의 샘플이 있다면 다수 클래스에서 10개의 샘플을 랜덤하게 선택하여 총 20개의 샘플을 데이터로 사용

In [ ]:
# 각 클래스의 샘플 인덱스를 추출합니다.
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# 각 클래스의 샘플 개수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# 클래스 0의 샘플만큼 클래스 1에서 중복을 허용하지 않고 랜덤하게 샘플을 뽑습니다.
# from class 1 without replacement
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

# 클래스 0의 타깃 벡터와 다운샘플링된 클래스 1의 타깃 벡터를 합칩니다.
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# 각 클래스의 샘플 인덱스를 추출합니다.
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# 각 클래스의 샘플 개수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

In [ ]:
n_class1

100

In [ ]:
np.vstack((features[i_class0,:], features[i_class1_downsampled,:]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

* 또 다른 방법은 소수의 클래스를 업샘플링하는 것
* 업샘플링에서는 다수 클래스의 샘플만큼 소수 클래스에서 중복을 허용하여 랜덤하게 샘플을 허용하여 랜덤하게 샘플 선택
* 결과적으로 다수 클래스와 소수 클래스의 샘플 수가 같아짐
* 업샘플링은 다운샘플링과 반대 방식으로 매우 비슷하게 구현

In [ ]:
# 클래스 1의 샘플 개수만큼 클래스 0에서 중복을 허용하여 랜덤하게 샘플을 선택합니다.
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# 클래스 0의 업샘플링된 타깃 벡터와 클래스 1의 타깃 벡터를 합칩니다.
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
# 클래스 0의 업샘플링된 특성 행렬과 클래스 1의 특성 행렬을 합칩니다.
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

array([[5. , 3.5, 1.6, 0.6],
       [4.5, 2.3, 1.3, 0.3],
       [5.1, 3.8, 1.9, 0.4],
       [5. , 3.3, 1.4, 0.2],
       [5. , 3.3, 1.4, 0.2]])

* 실전에는 불균형한 클래스가 아주 많음
* 대부분의 웹사이트 방문자는 구매 버튼을 클릭하지 않으며, 중대 질병인 암의 경우 종류가 다양하지만 상당수가 매우 희귀함
* 이런 이유 때문에 불균형 클래스를 다루는 일은 머신러닝에서 자주 발생


* 불균형 클래스 다루는 전략
1. 가장 좋은 방법은 소수 클래스의 샘플을 더 많이 모으는 것. 하지만 불가능한 경우가 많기 때문에 다른 선택 사항을 고려해야 함
2. 두번째 전략은 불균형한  클래스에 잘 맞는 모델 평가 지표를 사용
  * 정확도는 모델 성능을 평가하는데 자주 사용되는 평가 지표
  * 클래스가 불균형할 때 정확도는 잘 맞지 않음
  * 예를 들어 희귀한 암을 가진 샘플이 0.5%라면 아무도 암에 걸리지 않았다고 예측하는 단순한 모델도 99.5%의 정확도를 얻을 것임
  * 더 나은 지표로 오차 행렬, 정밀도, 재현율, F1점수, ROC곡선이 있음
3. 일부 모델에서 제공하는 클래스 가중치 매개변수 사용
  * 알고리즘이 불균형한 클래스를 조정할 수 있음
  * 사이킷런에 있는 많은 분류기들은 이에 적합한  class_weight 매개변수를 가지고 있음

4. 샘플링
  * 다운샘플링에서 소수 클래스 크기와 동일하게 다수 클래스의 랜덤한 부분집합 추출
  * 업샘플링에서는 다수 클래스 크기와 동일하게 소수 클래스로부터 중복을 허용하여 반복적으로 샘플 추출
  * 다운샘플링과 업샘플링 중 어떤 것을 사용할지 여부는 문제에 따라 달라짐
  * 일반적으로 두 전략을 모두 시도해보고 더 나은 결과를 내는 것을 선택

# 연습문제


* 'preprocessing.csv'파일은 온라인 쇼핑몰 고객들의 정보가 들어있다. Online Shopper인지 여부를 예측하기 위하여 머신러닝 알고리즘을 적용하려고 한다. 다음의 전처리 과정을 수행하여라. 
  1. 데이터프레임으로 csv파일을 불러와서 특성 행렬 X와 타깃 벡터 Y로 분리하라.
  2. 결측치에 대해 평균값으로 대체하라. 
  3. 범주형 특성에 대해 원핫인코딩하라. Y에 대해서는 정수레이블(0,1)로 변환하라.  
  4. 10개의 샘플 중 8개는 학습 데이터세트, 2개는 테스트세트로 하여 표준화 방법으로 특성행렬을 스케일링하라. 